<a href="https://colab.research.google.com/github/WolfgangNS/GTF-to-matrix/blob/main/Renn_Lab_TPM_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#courtesy of ChatGPT

#data pipeline:
#https://chatgpt.com/c/4c7769c8-c3a3-4562-b962-9ff1357a9bdd

#data download:
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5769613
#GSM5769613_SR02_merged.gtf.gz

#-----
#upload .gtf file to /content directory

### Download mappings (BioMart)

In [1]:
#made into Python variable so the query is easier to read :)
#https://useast.ensembl.org/biomart/martview

import pandas as pd
import urllib.parse

query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "hburtoni_paralog_ensembl_gene" />'
    '<Attribute name = "oniloticus_homolog_ensembl_gene" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

orthologs = pd.read_csv('genes.tsv',
                        sep='\t',
                        names=['hburtoni_paralog_ensembl_gene',
                                'oniloticus_homolog_ensembl_gene'])
orthologs.head(10)


wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22hburtoni_paralog_ensembl_gene%22%20/%3E%3CAttribute%20name%20%3D%20%22oniloticus_homolog_ensembl_gene%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes.tsv               [       <=>          ]  13.22M   983KB/s    in 13s     


,hburtoni_paralog_ensembl_gene,oniloticus_homolog_ensembl_gene
0,NaN,NaN
1,ENSHBUG00000023832,ENSONIG00000040832
2,ENSHBUG00000019623,ENSONIG00000040832
3,ENSHBUG00000016228,ENSONIG00000040832
4,ENSHBUG00000005138,ENSONIG00000040832
5,ENSHBUG00000016083,ENSONIG00000040832
6,ENSHBUG00000019072,ENSONIG00000040832
7,ENSHBUG00000008100,ENSONIG00000040832
8,ENSHBUG00000013528,ENSONIG00000040832
9,ENSHBUG00000012627,ENSONIG00000040832


In [2]:
#Again, but different
query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "ensembl_gene_id" />'
    '<Attribute name = "entrezgene_accession" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

ens_to_enz = pd.read_csv('genes2.tsv',
                        sep='\t',
                        names=['ensembl_gene_id',
                                'entrezgene_accession'])
ens_to_enz.head(10)


wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22ensembl_gene_id%22%20/%3E%3CAttribute%20name%20%3D%20%22entrezgene_accession%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes2.tsv              [             <=>    ] 622.62K  74.7KB/s    in 8.3s    


,ensembl_gene_id,entrezgene_accession
0,ENSHBUG00000023839,NaN
1,ENSHBUG00000000727,LOC102288583
2,ENSHBUG00000023838,hps1
3,ENSHBUG00000023837,LOC102292000
4,ENSHBUG00000023836,LOC102301706
5,ENSHBUG00000023835,slc25a29
6,ENSHBUG00000023834,LOC102298492
7,ENSHBUG00000023833,agap2
8,ENSHBUG00000023832,LOC102291978
9,ENSHBUG00000023831,LOC106633135


In [3]:
joined_df = pd.merge(ens_to_enz, orthologs, left_on='ensembl_gene_id', right_on='hburtoni_paralog_ensembl_gene', how='inner')
#TO-DO: I might need to change this to outer join

joined_df = joined_df.dropna()
joined_df = joined_df.drop(columns=['hburtoni_paralog_ensembl_gene'])

before_rows = len(ens_to_enz)
after_rows = len(joined_df)

print("Number of rows before joining:", before_rows)
print("Number of rows after joining and removing NaNs:", after_rows)
print("Preview of the resulting DataFrame:")
print(joined_df.head())

Number of rows before joining: 23922
Number of rows after joining and removing NaNs: 261369
Preview of the resulting DataFrame:
      ensembl_gene_id entrezgene_accession oniloticus_homolog_ensembl_gene
0  ENSHBUG00000000727         LOC102288583              ENSONIG00000040832
1  ENSHBUG00000000727         LOC102288583              ENSONIG00000035632
2  ENSHBUG00000000727         LOC102288583              ENSONIG00000008077
3  ENSHBUG00000000727         LOC102288583              ENSONIG00000041447
4  ENSHBUG00000000727         LOC102288583              ENSONIG00000013172


In [4]:
joined_df.to_csv("/content/entrez-ensembl-mappings.tsv", sep='\t')

### Download .tar (.gtf files inside)

In [ ]:
#https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/GSE192958_RAW.tar
!tar -zxvf GSE192958_RAW.tar -k

In [19]:
!gunzip *.gz

### Create gene count matrix from samples (HTSeq)

In [ ]:
!pip install HTSeq scanpy

In [ ]:
#15m for code below

In [50]:
import os
import HTSeq
import numpy as np
import pandas as pd
import time

# Directory containing .gtf files
directory = "/content"

# Function to extract gene expression counts from a single .gtf file
def extract_counts(gtf_file):
    counts = {}
    features = HTSeq.GFF_Reader(gtf_file)
    for feature in features:
        if feature.type == "transcript":
            gene_name = feature.attr.get("gene_name")
            ref_gene_name = feature.attr.get("ref_gene_name")
            tpm = feature.attr.get("TPM")  # Get TPM attribute
            if tpm is not None:
                tpm_value = float(tpm)
                if gene_name is not None:
                    counts[gene_name] = counts.get(gene_name, 0) + tpm_value
                elif ref_gene_name is not None:
                    counts[ref_gene_name] = counts.get(ref_gene_name, 0) + tpm_value
    return counts

# Iterate over .gtf files in the directory
all_counts = {}
for filename in os.listdir(directory):
    if filename.endswith(".gtf"):
        filepath = os.path.join(directory, filename)
        sample_name = os.path.splitext(filename)[0]
        counts = extract_counts(filepath)
        all_counts[sample_name] = counts

# Convert counts to DataFrame and save as TSV
gene_names = sorted(set().union(*(counts.keys() for counts in all_counts.values())))

sample_names = list(all_counts.keys())
count_matrix = np.zeros((len(sample_names), len(gene_names)), dtype=int)
for i, sample_name in enumerate(sample_names):
    counts = all_counts[sample_name]
    for j, gene_name in enumerate(gene_names):
        count_matrix[i, j] = counts.get(gene_name, 0)

count_df = pd.DataFrame(count_matrix, index=sample_names, columns=gene_names)
count_df.to_csv("/content/output.tsv", sep='\t')


####NOTE: THERE IS SOMETHING SERIOUSLY WRONG WITH THIS CODE. SOME COUNTS ARE REPORTED AT 1E+18, SOME IN THE NEGATIVE
###CHECK FLOATING POINT NUMBERS.

In [62]:
count_df.T.sort_values(by=['GSM5769628_SR19_merged'], ascending=False)

,GSM5769628_SR19_merged,GSM5769645_SR36_merged,GSM5769636_SR27_merged,GSM5769623_SR14_merged,GSM5769632_SR23_merged,GSM5769631_SR22_merged,GSM5769637_SR28_merged,GSM5769641_SR32_merged,GSM5769618_SR08_merged,GSM5769652_SR44_merged,...,GSM5769625_SR16_merged,GSM5769627_SR18_merged,GSM5769638_SR29_merged,GSM5769621_SR12_merged,GSM5769626_SR17_merged,GSM5769646_SR37_merged,GSM5769649_SR41_merged,GSM5769640_SR31_merged,GSM5769635_SR26_merged,GSM5769620_SR11_merged
LOC102297266,6440,4236,5928,4880,5403,5981,5565,2926,4407,2406,...,7754,3884,4519,4338,3920,4926,4877,4020,4701,4005
LOC102305530,6266,3164,4274,5018,3953,5013,4610,2629,4185,2581,...,5434,3992,3492,3501,3072,3864,3253,3808,3577,3353
snap25,5514,5558,7352,5759,7278,6654,7273,5113,6492,5242,...,7772,5348,7053,6379,5488,6691,5537,7633,6492,7849
LOC102309206,5443,3778,5659,4412,5311,4433,4798,3030,4159,3117,...,6230,3943,4375,3040,3910,4116,3489,4720,4048,4579
tmsb4x,4782,5143,5484,4043,5875,3827,5364,5101,5255,4567,...,5481,4106,7765,4638,4973,5292,4459,6055,5015,6289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC102309000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOC102308999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
LOC102308993,0,2,2,0,13,1,0,0,0,0,...,1,0,0,0,0,2,0,1,3,0
LOC102308989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
gene_mapping = dict(zip(joined_df["entrezgene_accession"], joined_df["oniloticus_homolog_ensembl_gene"]))
transposed_count_df = count_df.transpose()
transposed_count_df.rename(index=gene_mapping, inplace=True)
transposed_count_df = transposed_count_df.groupby(transposed_count_df.index).sum()
transposed_count_df = transposed_count_df.loc[~transposed_count_df.index.duplicated()]
transposed_count_df

,GSM5769628_SR19_merged,GSM5769645_SR36_merged,GSM5769636_SR27_merged,GSM5769623_SR14_merged,GSM5769632_SR23_merged,GSM5769631_SR22_merged,GSM5769637_SR28_merged,GSM5769641_SR32_merged,GSM5769618_SR08_merged,GSM5769652_SR44_merged,...,GSM5769625_SR16_merged,GSM5769627_SR18_merged,GSM5769638_SR29_merged,GSM5769621_SR12_merged,GSM5769626_SR17_merged,GSM5769646_SR37_merged,GSM5769649_SR41_merged,GSM5769640_SR31_merged,GSM5769635_SR26_merged,GSM5769620_SR11_merged
ENSONIG00000000013,7,4,11,5,9,5,9,3,6,7,...,6,6,8,5,5,5,10,8,5,8
ENSONIG00000000015,3,3,3,2,0,3,0,3,2,5,...,2,3,1,3,0,4,4,2,1,2
ENSONIG00000000019,2,2,0,2,1,1,2,3,2,4,...,1,3,0,2,4,3,3,1,2,0
ENSONIG00000000021,2,3,1,2,0,3,1,1,1,0,...,2,1,2,1,0,0,2,1,0,1
ENSONIG00000000047,22,24,40,20,33,26,31,19,32,26,...,27,26,45,31,24,31,23,43,28,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zswim7,7,8,11,6,12,7,16,4,11,9,...,13,6,9,8,11,8,4,16,13,14
zw10,3,1,3,1,1,1,1,3,2,1,...,2,2,1,2,2,2,3,2,2,2
zwilch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zzef1,17,11,14,16,13,16,12,15,15,14,...,11,17,10,11,14,13,12,10,14,13


In [59]:
transposed_count_df.sort_values(by=['GSM5769628_SR19_merged'], ascending=False)

,GSM5769628_SR19_merged,GSM5769645_SR36_merged,GSM5769636_SR27_merged,GSM5769623_SR14_merged,GSM5769632_SR23_merged,GSM5769631_SR22_merged,GSM5769637_SR28_merged,GSM5769641_SR32_merged,GSM5769618_SR08_merged,GSM5769652_SR44_merged,...,GSM5769625_SR16_merged,GSM5769627_SR18_merged,GSM5769638_SR29_merged,GSM5769621_SR12_merged,GSM5769626_SR17_merged,GSM5769646_SR37_merged,GSM5769649_SR41_merged,GSM5769640_SR31_merged,GSM5769635_SR26_merged,GSM5769620_SR11_merged
LOC102297266,6440,4236,5928,4880,5403,5981,5565,2926,4407,2406,...,7754,3884,4519,4338,3920,4926,4877,4020,4701,4005
LOC102305530,6266,3164,4274,5018,3953,5013,4610,2629,4185,2581,...,5434,3992,3492,3501,3072,3864,3253,3808,3577,3353
snap25,5514,5558,7352,5759,7278,6654,7273,5113,6492,5242,...,7772,5348,7053,6379,5488,6691,5537,7633,6492,7849
LOC102309206,5443,3778,5659,4412,5311,4433,4798,3030,4159,3117,...,6230,3943,4375,3040,3910,4116,3489,4720,4048,4579
tmsb4x,4782,5143,5484,4043,5875,3827,5364,5101,5255,4567,...,5481,4106,7765,4638,4973,5292,4459,6055,5015,6289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC102309000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LOC102308999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
LOC102308993,0,2,2,0,13,1,0,0,0,0,...,1,0,0,0,0,2,0,1,3,0
LOC102308989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
transposed_count_df.to_csv("/content/matrix.tsv", sep='\t')

### Phenotype data

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/miniml/GSE192958_family.xml.tgz
!tar -zxvf GSE192958_family.xml.tgz

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('GSE192958_family.xml')
root = tree.getroot()
with open('attributes.tsv', 'w') as tsvfile:
    tsvfile.write('GSM_id\tsamplename\n')
    for sample in root.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Sample'):
        sample_id = sample.attrib['iid']
        for subsample in sample.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Title'):
            # print(subsample.text)
            # print(sample.attrib['iid'])
            GSM_id = subsample.text
            samplename = sample.attrib['iid']
            tsvfile.write(f'{GSM_id}\t{samplename}\n')


In [ ]:
!head -n 10 attributes.tsv

### (Miscellaneous)

In [ ]:
#Briliant solution: https://unix.stackexchange.com/a/22624
!tail -n 420 GSE192958_series_matrix.txt \
  | head -30